# Image Extractor - Standalone Pipeline

Extract images, charts, and diagrams from PDFs using Vision LLM.

**Features:**
- Uses `pdf2image` (FREE) to render PDF pages
- Sends to Vision LLM to detect image regions
- Crops and saves detected images
- Generates descriptions for each image
- Outputs `image_extractions.json`

**No Google Doc AI / Layout Parser dependencies.**

---

## How It Works

1. **Render**: Convert each PDF page to an image using Poppler
2. **Detect**: Send page to Vision LLM → get bounding boxes of images/charts
3. **Crop**: Extract detected regions as separate image files
4. **Describe**: Get text descriptions for each detected image
5. **Output**: Save JSON with image metadata + descriptions


## 1. Install Dependencies


In [ ]:
# Install Python packages
%pip install -q pdf2image Pillow openai

# Install Poppler (required by pdf2image to render PDFs)
!apt-get update -qq && apt-get install -y -qq poppler-utils

print("✅ Dependencies installed successfully!")


## 2. Configuration


In [ ]:
import os

# ============================================================
# CONFIGURATION - Update these values
# ============================================================

# OpenAI API Key (for Vision LLM)
OPENAI_API_KEY = ""  # Paste your key here, or set via environment

# Output directory for extracted images
OUTPUT_DIR = "image_output"

# ============================================================
# Set environment variable
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
elif not os.getenv("OPENAI_API_KEY"):
    print("⚠️ Warning: OPENAI_API_KEY not set. Please set it above or via environment.")

# Create output directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "images"), exist_ok=True)

print(f"✅ Output directory: {OUTPUT_DIR}/")
print(f"✅ Images will be saved to: {OUTPUT_DIR}/images/")


## 3. Upload PDF


In [ ]:
from google.colab import files

print("Upload a PDF file to extract images from...")
uploaded = files.upload()
pdf_filename = list(uploaded.keys())[0]
print(f"✅ Uploaded: {pdf_filename}")


## 4. Image Extractor Class

Core logic for detecting and extracting images from PDF pages.


In [ ]:
import json
import base64
import re
from typing import List, Dict, Any, Optional
from pdf2image import convert_from_path
from PIL import Image
import io
from openai import OpenAI
from datetime import datetime


class ImageExtractor:
    """
    Extracts images, charts, and diagrams from PDFs using Vision LLM.
    
    Pipeline:
    1. Render PDF pages as images (pdf2image + Poppler)
    2. Send each page to Vision LLM to detect image regions
    3. Crop detected regions and save to disk
    4. Return structured JSON with image metadata + descriptions
    """
    
    def __init__(self, output_dir: str = "image_output", api_key: Optional[str] = None):
        """
        Args:
            output_dir: Directory to save extracted images
            api_key: OpenAI API key (uses env var if not provided)
        """
        self.output_dir = output_dir
        self.images_dir = os.path.join(output_dir, "images")
        
        # Create directories
        os.makedirs(self.output_dir, exist_ok=True)
        os.makedirs(self.images_dir, exist_ok=True)
        
        # Initialize OpenAI client
        api_key = api_key or os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("OpenAI API key required. Set OPENAI_API_KEY or pass api_key parameter.")
        self.client = OpenAI(api_key=api_key)
        
        # Counter for image IDs
        self.image_counter = 0
    
    def extract(self, pdf_path: str) -> Dict[str, Any]:
        """
        Main entry point - extract all images from a PDF.
        
        Args:
            pdf_path: Path to the PDF file
            
        Returns:
            Dictionary with metadata and list of extracted images
        """
        print(f"📄 Processing: {pdf_path}")
        
        # 1. Render PDF pages as images
        print("🔄 Rendering PDF pages...")
        page_images = self._render_pages(pdf_path)
        print(f"   ✅ Rendered {len(page_images)} pages")
        
        # 2. Process each page to detect images
        all_images = []
        for page_num, page_image in enumerate(page_images, start=1):
            print(f"🔍 Analyzing page {page_num}/{len(page_images)}...")
            detected = self._detect_images_on_page(page_image, page_num)
            all_images.extend(detected)
            print(f"   Found {len(detected)} image(s) on page {page_num}")
        
        # 3. Build result
        result = {
            "metadata": {
                "source_file": os.path.basename(pdf_path),
                "page_count": len(page_images),
                "images_found": len(all_images),
                "extraction_timestamp": datetime.now().isoformat()
            },
            "images": all_images
        }
        
        print(f"\n✅ Extraction complete! Found {len(all_images)} images across {len(page_images)} pages.")
        return result
    
    def _render_pages(self, pdf_path: str) -> List[Image.Image]:
        """Convert PDF pages to PIL Images using pdf2image."""
        try:
            images = convert_from_path(pdf_path, dpi=150)
            return images
        except Exception as e:
            print(f"❌ Error rendering PDF: {e}")
            print("   Make sure Poppler is installed: apt-get install -y poppler-utils")
            raise
    
    def _detect_images_on_page(self, page_image: Image.Image, page_num: int) -> List[Dict[str, Any]]:
        """
        Send page to Vision LLM to detect image regions and get descriptions.
        
        Returns list of detected images with bounding boxes and descriptions.
        """
        # Convert PIL Image to base64
        img_bytes = io.BytesIO()
        page_image.save(img_bytes, format='PNG')
        img_bytes.seek(0)
        base64_image = base64.b64encode(img_bytes.read()).decode('utf-8')
        
        # Build detection prompt
        prompt = """Analyze this PDF page and identify ALL visual elements that are NOT plain text.

Look for:
- Charts (bar, line, pie, etc.)
- Diagrams and flowcharts
- Infographics
- Photographs or illustrations
- Tables with visual formatting
- Icons or logos (if significant)
- QR codes

For EACH visual element found, provide:
1. bbox: Bounding box as [x_min, y_min, x_max, y_max] where values are percentages (0.0 to 1.0) of page width/height
2. type: One of "chart", "diagram", "infographic", "photo", "table", "icon", "qr_code", "other"
3. description: Brief description of what the visual shows (1-2 sentences)

Respond in this exact JSON format:
{
  "visuals": [
    {
      "bbox": [0.1, 0.2, 0.5, 0.6],
      "type": "chart",
      "description": "Bar chart showing quarterly revenue growth from Q1-Q4 2024"
    }
  ]
}

If NO visual elements are found, respond with:
{"visuals": []}

IMPORTANT: Only include actual images/visuals. Do NOT include plain text paragraphs or headings."""

        try:
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/png;base64,{base64_image}",
                                    "detail": "high"
                                }
                            }
                        ]
                    }
                ],
                max_tokens=1000,
                temperature=0.1
            )
            
            # Parse response
            content = response.choices[0].message.content.strip()
            
            # Extract JSON from response (handle markdown code blocks)
            json_match = re.search(r'\{[\s\S]*\}', content)
            if json_match:
                parsed = json.loads(json_match.group())
                visuals = parsed.get("visuals", [])
            else:
                print(f"   ⚠️ Could not parse response: {content[:100]}...")
                visuals = []
            
            # Process each detected visual
            detected_images = []
            for visual in visuals:
                bbox = visual.get("bbox", [])
                if len(bbox) != 4:
                    continue
                
                # Crop and save the image
                self.image_counter += 1
                image_id = f"img_{self.image_counter:03d}"
                file_path = self._crop_and_save(page_image, bbox, page_num, image_id)
                
                detected_images.append({
                    "id": image_id,
                    "page": page_num,
                    "type": visual.get("type", "unknown"),
                    "bbox": bbox,
                    "file_path": file_path,
                    "description": visual.get("description", "")
                })
            
            return detected_images
            
        except Exception as e:
            print(f"   ❌ Error detecting images on page {page_num}: {e}")
            return []
    
    def _crop_and_save(self, page_image: Image.Image, bbox: List[float], 
                       page_num: int, image_id: str) -> str:
        """
        Crop the detected region from the page image and save to disk.
        
        Args:
            page_image: Full page as PIL Image
            bbox: [x_min, y_min, x_max, y_max] as percentages (0.0-1.0)
            page_num: Page number
            image_id: Unique image identifier
            
        Returns:
            Path to saved image file
        """
        width, height = page_image.size
        
        # Convert normalized coordinates to pixels
        x_min = int(bbox[0] * width)
        y_min = int(bbox[1] * height)
        x_max = int(bbox[2] * width)
        y_max = int(bbox[3] * height)
        
        # Ensure valid crop box
        x_min = max(0, x_min)
        y_min = max(0, y_min)
        x_max = min(width, x_max)
        y_max = min(height, y_max)
        
        if x_max <= x_min or y_max <= y_min:
            print(f"   ⚠️ Invalid bbox for {image_id}, skipping crop")
            return ""
        
        # Crop
        cropped = page_image.crop((x_min, y_min, x_max, y_max))
        
        # Save
        filename = f"page{page_num}_{image_id}.png"
        save_path = os.path.join(self.images_dir, filename)
        cropped.save(save_path, format='PNG')
        
        return save_path


print("✅ ImageExtractor class defined.")


## 5. Run Extraction


In [ ]:
# Initialize extractor and run
extractor = ImageExtractor(output_dir=OUTPUT_DIR)
result = extractor.extract(pdf_filename)

# Preview result summary
print("\n" + "="*50)
print("EXTRACTION SUMMARY")
print("="*50)
print(f"Source: {result['metadata']['source_file']}")
print(f"Pages: {result['metadata']['page_count']}")
print(f"Images Found: {result['metadata']['images_found']}")
print("="*50)


## 6. Preview Extracted Images


In [ ]:
from IPython.display import display, Image as IPImage

if result["images"]:
    print(f"Showing {len(result['images'])} extracted images:\n")
    
    for img_data in result["images"]:
        print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"📸 {img_data['id']} | Page {img_data['page']} | Type: {img_data['type']}")
        print(f"📝 {img_data['description']}")
        print(f"📁 {img_data['file_path']}")
        
        # Display the image
        if img_data['file_path'] and os.path.exists(img_data['file_path']):
            display(IPImage(filename=img_data['file_path'], width=400))
        print()
else:
    print("No images were detected in this PDF.")


## 7. Save JSON Output


In [ ]:
# Save JSON output
output_json_path = os.path.join(OUTPUT_DIR, "image_extractions.json")

with open(output_json_path, "w") as f:
    json.dump(result, f, indent=2)

print(f"✅ JSON saved to: {output_json_path}")

# Display JSON content
print("\n📄 JSON Content Preview:")
print("-" * 50)
print(json.dumps(result, indent=2)[:2000])
if len(json.dumps(result, indent=2)) > 2000:
    print("\n... (truncated)")


## 8. Download Results


In [ ]:
# Download JSON
files.download(output_json_path)

# Zip and download images if any were extracted
if result["images"]:
    import shutil
    zip_path = os.path.join(OUTPUT_DIR, "extracted_images")
    shutil.make_archive(zip_path, 'zip', extractor.images_dir)
    print(f"✅ Images zipped to: {zip_path}.zip")
    files.download(f"{zip_path}.zip")
else:
    print("No images to download.")
